In [1]:
import pandas as pd
# import pymysql
import sys
import os

In [2]:
# # OP.GG Database에 접근하기 위한 연결 객체 생성
# con = pymysql.connect(
#     user = os.environ['LOL_KR_ID'],
#     passwd = os.environ['LOL_KR_PW'],
#     host = os.environ['LOL_KR_HOST'],
#     db = 'lol',
#     charset = 'utf8'
# )
# cursor = con.cursor(pymysql.cursors.DictCursor)

In [3]:
# # 데이터 구조 파악을 위한 sample data fetch
# # SQL
# cursor.execute('''
# SELECT STRAIGHT_JOIN *
# FROM opGame o FORCE INDEX (ix_createDate),
# p_opGameStats p FORCE INDEX (`PRIMARY`)
# WHERE o.gameId = p.gameId
# AND o.createDate >= '2021-10-20'
# AND p.createDate >= '2021-10-20'
# ORDER BY o.createDate DESC
# LIMIT 3000
# ''')
# sample = cursor.fetchall()

# # DataFrame으로 변환
# sample = pd.DataFrame(sample)

# # 저장
# sample.to_csv("sample.csv", mode='w')

In [4]:
# 미리 뽑아둔 CSV 파일에서 가져오기
sample = pd.read_csv('sample.csv', index_col=0)

FileNotFoundError: ignored

In [ ]:
# 데이터 확인
sample

In [ ]:
# Column 수가 많아서 print했을 때 다 보이게 옵션 설정
pd.set_option('display.max_columns', None)

In [ ]:
# columns 확인
print(sample.columns)

# 값이 정상적인지 보기 위해 describe로 요약 정보 출력
sample.describe(include='all',datetime_is_numeric=True)

In [ ]:
# 필요한 값만 선택
sample = sample[['gameId','subType','championId','result','position']]

In [ ]:
# 최종적으로 만들어진 sample dataframe
sample

In [ ]:
# Reset sample data
%reset

In [ ]:
import pandas as pd
import numpy as np
# import pymysql
import sys
import os
from datetime import date, timedelta
from scipy.stats import norm
import requests

In [ ]:
# # OP.GG Database에 접근하기 위한 연결 객체 생성
# con = pymysql.connect(
#     user = os.environ['LOL_KR_ID'],
#     passwd = os.environ['LOL_KR_PW'],
#     host = os.environ['LOL_KR_HOST'],
#     db = 'lol',
#     charset = 'utf8'
# )
# cursor = con.cursor(pymysql.cursors.DictCursor)

In [ ]:
# # 패치 날짜 데이터 불러오기
# cursor.execute('''
# SELECT version, date
# FROM lolVersionHistory
# ''')
# patchDate = cursor.fetchall()

# patchDate = pd.DataFrame(patchDate)

# # version => season, patch로 가공, 세부 버전(핫픽스, 밸런싱 X 패치 등) 제거
# patchDate['season'] = pd.to_numeric(patchDate['version'].str.split('.').str[0])
# patchDate['patch'] = pd.to_numeric(patchDate['version'].str.split('.').str[1])
# patchDate = patchDate.groupby(['season','patch'],as_index=False).min('date').drop('version', axis = 1).sort_values(['season','patch'])

In [ ]:
# last_patch_start = patchDate.iloc[-2].date
# last_patch_end = patchDate.iloc[-1].date

In [ ]:
# bot_count = pd.DataFrame()
# spt_count = pd.DataFrame()
# duo_count = pd.DataFrame()

# for date in pd.date_range(last_patch_start,last_patch_end):
    
#     if date.strftime("%Y-%m-%d") == last_patch_start.strftime("%Y-%m-%d"):
#         date_start = last_patch_start.strftime("%Y-%m-%d %H:%M:%S")
#     else:
#         date_start = date.strftime("%Y-%m-%d 00:00:00")
        
#     if date.strftime("%Y-%m-%d") == last_patch_end.strftime("%Y-%m-%d"):
#         date_end = last_patch_end.strftime("%Y-%m-%d %H:%M:%S")
#     else:
#         date_end = (date+timedelta(1)).strftime("%Y-%m-%d 00:00:00")
        
#     # BOT data fetch
#     # SQL
#     cursor.execute(f'''
#     SELECT STRAIGHT_JOIN o.gameId, tierRank, championId, result
#     FROM opGame o FORCE INDEX (ix_createDate),
#     p_opGameStats p FORCE INDEX (`PRIMARY`)
#     WHERE o.gameId = p.gameId
#     AND o.createDate >= '{date_start}'
#     AND o.createDate < '{date_end}'
#     AND p.createDate >= '{date_start}'
#     AND p.createDate < '{date_end}'
#     AND position ='A'
#     AND result != 3
#     AND subType = 420
#     ''')
#     botstats = cursor.fetchall()
#     botstats = pd.DataFrame(botstats)
#     botstats['tierRank'] = botstats['tierRank'].str[0]
#     botstats = botstats[botstats.tierRank.isin(['P','D','M','R','C'])]
    
#     # SPT data fetch
#     # SQL
#     cursor.execute(f'''
#     SELECT STRAIGHT_JOIN o.gameId, tierRank, championId, result
#     FROM opGame o FORCE INDEX (ix_createDate),
#     p_opGameStats p FORCE INDEX (`PRIMARY`)
#     WHERE o.gameId = p.gameId
#     AND o.createDate >= '{date_start}'
#     AND o.createDate < '{date_end}'
#     AND p.createDate >= '{date_start}'
#     AND p.createDate < '{date_end}'
#     AND position ='S'
#     AND result != 3
#     AND subType = 420
#     ''')
#     sptstats = cursor.fetchall()
#     sptstats = pd.DataFrame(sptstats)
#     sptstats['tierRank'] = sptstats['tierRank'].str[0]
#     sptstats = sptstats[sptstats.tierRank.isin(['P','D','M','R','C'])]
    
#     # bot count
#     bot_count_new = botstats.groupby(['championId'],as_index=False).\
#     apply(lambda x: pd.Series({'win_count':sum(x['result'] == 'WIN'),
#                                'count':len(x)}))
    
#     # spt count
#     spt_count_new = sptstats.groupby(['championId'],as_index=False).\
#     apply(lambda x: pd.Series({'win_count':sum(x['result'] == 'WIN'),
#                                'count':len(x)}))
    
#     # duo count
#     duo_count_new = botstats.merge(sptstats,
#                                how = 'inner',
#                                on = ['gameId','result']).\
#     groupby(['championId_x','championId_y'],
#             as_index=False).\
#     apply(lambda x: pd.Series({'win_count':sum(x['result'] == 'WIN'),
#                                'count':len(x)}))
    
#     # concatenate
#     bot_count = pd.concat([bot_count, bot_count_new])
#     spt_count = pd.concat([spt_count, spt_count_new])
#     duo_count = pd.concat([duo_count, duo_count_new])

In [ ]:
# bot_count = bot_count.groupby(['championId'],as_index=False).\
# apply(lambda x: pd.Series({'WR':sum(x['win_count'])/sum(x['count']),
#                            'count':sum(x['count'])}))
# spt_count = spt_count.groupby(['championId'],as_index=False).\
# apply(lambda x: pd.Series({'WR':sum(x['win_count'])/sum(x['count']),
#                            'count':sum(x['count'])}))
# duo_count = duo_count.groupby(['championId_x','championId_y'],as_index=False).\
# apply(lambda x: pd.Series({'WR':sum(x['win_count'])/sum(x['count']),
#                            'count':sum(x['count'])}))

In [ ]:
# # 저장
# bot_count.to_csv("bot_count.csv", mode='w')
# spt_count.to_csv("spt_count.csv", mode='w')
# duo_count.to_csv("duo_count.csv", mode='w')

In [ ]:
# 미리 뽑아둔 CSV 파일에서 가져오기
bot_count = pd.read_csv('bot_count.csv', index_col=0)
spt_count = pd.read_csv('spt_count.csv', index_col=0)
duo_count = pd.read_csv('duo_count.csv', index_col=0)

In [ ]:
# 픽률 0.5% 이상인 챔피언만 남기기
bot_count = bot_count[bot_count['count'] >= 0.005*sum(bot_count['count'])/2]
spt_count = spt_count[spt_count['count'] >= 0.005*sum(spt_count['count'])/2]

In [ ]:
# 매치업 50게임 이상인 조합만 남기기
duo_count = duo_count[duo_count['count'] >= 50]

In [ ]:
# DataFrame 병합
duo_count = duo_count.merge(bot_count,
                            how = 'inner',
                            left_on = 'championId_x',
                            right_on = 'championId',
                            suffixes = ('', '_BOT')
                           ).\
                      merge(spt_count,
                            how = 'inner',
                            left_on = 'championId_y',
                            right_on = 'championId',
                            suffixes = ('', '_SPT'))

In [ ]:
duo_count = duo_count[['championId_x','championId_y','WR','count','WR_BOT','count_BOT','WR_SPT','count_SPT']]

In [ ]:
duo_count

In [ ]:
duo_count.sort_values('WR',ascending=False)

In [ ]:
duo_count['score_SPT'] = norm.cdf((duo_count['WR']-duo_count['WR_BOT'])/
                                  np.sqrt((duo_count['WR']*(1-duo_count['WR']))/duo_count['count']+
                                          (duo_count['WR_BOT']*(1-duo_count['WR_BOT']))/duo_count['count_BOT']))
duo_count['score_BOT'] = norm.cdf((duo_count['WR']-duo_count['WR_SPT'])/
                                  np.sqrt((duo_count['WR']*(1-duo_count['WR']))/duo_count['count']+
                                          (duo_count['WR_SPT']*(1-duo_count['WR_SPT']))/duo_count['count_SPT']))
duo_count['score'] = norm.cdf((duo_count['WR']-0.5)/
                              np.sqrt((duo_count['WR']*(1-duo_count['WR']))/duo_count['count']))

In [ ]:
duo_count.sort_values('score',ascending=False)

In [ ]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [ ]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [ ]:
champion_df

In [ ]:
# 챔피언 key -> name
duo_count = duo_count.merge(champion_df,
                            how = 'inner',
                            left_on = 'championId_x',
                            right_on = 'key').\
                      merge(champion_df,
                            how = 'inner',
                            left_on = 'championId_y',
                            right_on = 'key',
                            suffixes = ('_BOT','_SPT')
                           )

In [ ]:
# 필요한 컬럼만 취합
result = duo_count[['name_BOT','name_SPT','WR','count','score','score_BOT','score_SPT']].\
sort_values('score',ascending=False)

In [ ]:
result